## Deploy the application on the Production Centre

This Jupyter Notebook creates a Web Processing Service request on the Ellip Platform Deployer to deploy the data transformation application on the Production Centre. 

* Set the deployer

In [7]:
deployer = 'https://ec-ecopotential-apps-deployer.terradue.com'

* Define the data transformation application information

In [3]:
app = dict([('artifact_id', 'dcs-s2-ndvi-boa'),
            ('version', '1.9'),
            ('repository', 'Github Organizations'),
            ('community', 'ec-ecopotential')])

* First do the imports of the Python libraries required

In [2]:
import jenkins
import time
import lxml.etree as etree
import sys
import requests
import os
import string
import hashlib
import urllib2
import pytz
from datetime import datetime
import time
import owslib
from owslib.wps import monitorExecution
import uuid
from owslib.wps import WebProcessingService

* Connect to the WPS server and do a GetCapabilities request to list the available process:

In [8]:
wps_url = '%s/zoo-bin/zoo_loader.cgi' % deployer

wps = WebProcessingService(wps_url, verbose=False, skip_caps=True)

wps.getcapabilities()

In [9]:
for index, elem in enumerate(wps.processes):
    print(index, elem.identifier)

(0, 'ec_ecopotential_dcs_s1_temporal_series_dcs_s1_temporal_series_0_3')
(1, 'ec_ecopotential_dcs_s2_crop_s2_crop_1_9')
(2, 'coordinator_ec_ecopotential_dcs_s2_crop_s2_crop_1_9')
(3, 'ec_ecopotential_dcs_s2_ndvi_boa_ndvi_boa_1_9')
(4, 'ec_ecopotential_dcs_snap_hydroperiod_preproc_dcs_snap_hydroperiod_preproc_0_6')
(5, 'GetStatus')


* Select the **DeployProcess** process to submit a DescribeProcess request and list the inputs:

In [23]:
process_id = 'DeployProcess'
        
process = wps.describeprocess(process_id)

for input in process.dataInputs:
    print(input.identifier)

applicationPackage


* Build the Python dictionary with the inputs:

In [10]:
docker_hub = 'https://store.terradue.com'

ows_context_url = '%s/api/%s/_applications/%s/%s/%s/%s-%s-context.xml' % (docker_hub,
                                                                          app['community'], 
                                                                   app['community'],  
                                                                   app['artifact_id'], 
                                                                   app['version'],  
                                                                   app['artifact_id'], 
                                                                   app['version'])
    

In [11]:
ows_context_url

'https://store.terradue.com/api/ec-ecopotential/_applications/ec-ecopotential/dcs-s2-ndvi-boa/1.9/dcs-s2-ndvi-boa-1.9-context.xml'

In [26]:
inputs = [('applicationPackage', ows_context_url)]

* Submit the Execute WPS request:

In [19]:
execution = owslib.wps.WPSExecution(url=wps.url)

execution_request = execution.buildRequest(process_id, 
                                           inputs, 
                                           output = [('deployResult', False)])
        
execution_response = execution.submitRequest(etree.tostring(execution_request))
        
execution.parseResponse(execution_response)

* Monitor the request:

In [20]:
monitorExecution(execution)

 owslib.wps.WPSException : {'locator': None, 'code': 't2h5', 'text': '{\n  "errors" : [ {\n    "status" : 401,\n    "message" : "Unauthorized"\n  } ]\n}'}


* Check if the application was successfully deployed

In [21]:
if execution.isSucceded(): 
            
    print 'Application deployed!'
            
else: 

    print 'Application not deployed :-('

Application not deployed :-(
